In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 预处理数据: 转换为 1 维，类型转换为 float32，特征值缩放到 [0, 1]
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# 从训练集中分出 1000 个样本用作验证
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [4]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # loss 函数
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # 记录的 metric 列表
    metrics=[keras.metrics.SparseCategoricalAccuracy(), ]
)

In [5]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # 验证集，每个 epoch 后查看模型在验证集上的 loss 和 metrics
    validation_data=(x_val, y_val)
)

Fit model on training data
Epoch 1/2
782/782 [==============================] - 6s 6ms/step - loss: 0.3430 - sparse_categorical_accuracy: 0.9034 - val_loss: 0.2571 - val_sparse_categorical_accuracy: 0.9186
Epoch 2/2
782/782 [==============================] - 4s 5ms/step - loss: 0.1590 - sparse_categorical_accuracy: 0.9523 - val_loss: 0.1434 - val_sparse_categorical_accuracy: 0.9563


In [6]:
history.history

{'loss': [0.3430411219596863, 0.1589580923318863],
 'sparse_categorical_accuracy': [0.9034000039100647, 0.9523400068283081],
 'val_loss': [0.2571413516998291, 0.14339785277843475],
 'val_sparse_categorical_accuracy': [0.9186000227928162, 0.9563000202178955]}

In [7]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
79/79 [==============================] - 0s 4ms/step - loss: 0.1423 - sparse_categorical_accuracy: 0.9585
test loss, test acc: [0.14227981865406036, 0.9585000276565552]
Generate predictions for 3 samples
1/1 [==============================] - 0s 60ms/step
predictions shape: (3, 10)


In [8]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [9]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

In [10]:
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred))


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 3s 3ms/step - loss: 0.0159


In [11]:
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 3s 3ms/step - loss: 0.0390


In [12]:
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        # 创建状态变量 true_positives
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # 更新状态变量
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_state(self):
        # 在每个 epoch 开始指标的状态被重置
        self.true_positives.assign(0.0)


model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
782/782 [==============================] - 5s 6ms/step - loss: 0.3395 - categorical_true_positives: 45154.0000
Epoch 2/3
782/782 [==============================] - 4s 6ms/step - loss: 0.1592 - categorical_true_positives: 47583.0000
Epoch 3/3
782/782 [==============================] - 5s 6ms/step - loss: 0.1180 - categorical_true_positives: 48225.0000


In [13]:
x = tf.constant([1.8, 2.2], dtype=tf.float32)
tf.cast(x, tf.int32)


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2])>

In [14]:
x = tf.constant([1.8, 2.2], dtype=tf.float32)
tf.dtypes.cast(x, tf.int32)


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2])>

In [15]:
A = tf.constant([2, 20, 30, 3, 6])
tf.math.argmax(A)  # A[2] 是张量 A 中最大值

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [16]:
B = tf.constant([[2, 20, 30, 3, 6],
                 [3, 11, 16, 1, 8],
                 [14, 45, 23, 5, 27]])

In [17]:
tf.math.argmax(B, 0)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([2, 2, 0, 2, 2], dtype=int64)>

In [18]:
tf.math.argmax(B, 1)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2, 2, 1], dtype=int64)>

In [19]:
C = tf.constant([0, 0, 0, 0])

In [20]:
tf.math.argmax(C)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [21]:
t1 = [[1, 2, 3],
      [4, 5, 6]]

In [22]:
print(tf.shape(t1).numpy())

[2 3]


In [23]:
t2 = tf.reshape(t1, [6])

In [24]:
t2

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6])>

In [25]:
tf.reshape(t2, [3, 2])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]])>

In [26]:
tf.reshape([1, 2, 3], [2, 2])

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 3 values, but the requested shape has 4 [Op:Reshape]

In [27]:
t = [[1, 2, 3],
     [4, 5, 6]]
tf.reshape(t, [3, 2]).numpy()

array([[1, 2],
       [3, 4],
       [5, 6]])

In [28]:
tf.transpose(t, perm=[1, 0]).numpy()

array([[1, 4],
       [2, 5],
       [3, 6]])

In [29]:
t = [[1, 2, 3],
     [4, 5, 6]]
tf.reshape(t, [-1])

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6])>

In [30]:
tf.reshape(t, [3, -1])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]])>

In [31]:
tf.reshape(t, [-1, 2])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]])>

In [32]:
tf.reshape([7], []).numpy()


7

In [33]:
t = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(tf.shape(t).numpy())

[9]


In [34]:
tf.reshape(t, [3, 3])

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])>

In [35]:
t = [[[1, 1], [2, 2]],
     [[3, 3], [4, 4]]]
print(tf.shape(t).numpy())

tf.reshape(t, [2, 4])




[2 2 2]


<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[1, 1, 2, 2],
       [3, 3, 4, 4]])>

In [36]:
t = [[[1, 1, 1],
      [2, 2, 2]],
     [[3, 3, 3],
      [4, 4, 4]],
     [[5, 5, 5],
      [6, 6, 6]]]
print(tf.shape(t).numpy())

[3 2 3]


In [37]:
tf.reshape(t, [-1])

<tf.Tensor: shape=(18,), dtype=int32, numpy=array([1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6])>

In [38]:
tf.reshape(t, [2, -1])

<tf.Tensor: shape=(2, 9), dtype=int32, numpy=
array([[1, 1, 1, 2, 2, 2, 3, 3, 3],
       [4, 4, 4, 5, 5, 5, 6, 6, 6]])>

In [39]:
tf.reshape(t, [-1, 9])

<tf.Tensor: shape=(2, 9), dtype=int32, numpy=
array([[1, 1, 1, 2, 2, 2, 3, 3, 3],
       [4, 4, 4, 5, 5, 5, 6, 6, 6]])>

In [40]:
tf.reshape(t, [2, -1, 3])

<tf.Tensor: shape=(2, 3, 3), dtype=int32, numpy=
array([[[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]],

       [[4, 4, 4],
        [5, 5, 5],
        [6, 6, 6]]])>